## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-22-13-06-29 +0000,bbc,Hen cages and pig farrowing crates face ban,https://www.bbc.com/news/articles/c93wxd27dvko...
1,2025-12-22-13-01-00 +0000,startribune,Ramstad: Picking up what’s left of UCare is th...,https://www.startribune.com/medica-ucare-healt...
2,2025-12-22-13-00-00 +0000,missionlocal,What went wrong in Ward 86? The hours and minu...,https://missionlocal.org/2025/12/sf-hospital-k...
3,2025-12-22-12-59-57 +0000,nyt,"As Trump Clings to Tariffs, His Argentine Ally...",https://www.nytimes.com/2025/12/22/world/ameri...
4,2025-12-22-12-59-21 +0000,nyt,"Angering Denmark, Trump Appoints Special Envoy...",https://www.nytimes.com/2025/12/22/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2391/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,19
9,up,11
334,was,10
108,bondi,9
261,epstein,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
180,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...,61
36,2025-12-22-11-29-46 +0000,wapo,Car bomb in Moscow kills senior Russian genera...,https://www.washingtonpost.com/world/2025/12/2...,55
183,2025-12-21-20-10-45 +0000,nypost,Trump’s No. 2 at DOJ Todd Blanche emphatic Eps...,https://nypost.com/2025/12/21/us-news/trumps-n...,47
191,2025-12-21-19-18-08 +0000,nypost,Nicki Minaj makes surprise appearance with Eri...,https://nypost.com/2025/12/21/us-news/nicki-mi...,47
185,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...,45


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
180,61,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...
36,55,2025-12-22-11-29-46 +0000,wapo,Car bomb in Moscow kills senior Russian genera...,https://www.washingtonpost.com/world/2025/12/2...
183,47,2025-12-21-20-10-45 +0000,nypost,Trump’s No. 2 at DOJ Todd Blanche emphatic Eps...,https://nypost.com/2025/12/21/us-news/trumps-n...
205,39,2025-12-21-17-30-34 +0000,nypost,US military targeting third tanker off coast o...,https://nypost.com/2025/12/21/world-news/us-mi...
200,38,2025-12-21-17-57-50 +0000,latimes,'Avatar: Fire and Ash' heats up the box office...,https://www.latimes.com/entertainment-arts/bus...
100,38,2025-12-22-06-38-22 +0000,nypost,Police claim Bondi Beach gunman Naveed Akram r...,https://nypost.com/2025/12/22/world-news/polic...
185,37,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...
80,32,2025-12-22-09-34-12 +0000,nypost,New York parents say kids ‘freeze’ on mandated...,https://nypost.com/2025/12/22/business/new-yor...
210,26,2025-12-21-16-08-05 +0000,nypost,Russia claims talks on US peace plan for Ukrai...,https://nypost.com/2025/12/21/world-news/russi...
103,25,2025-12-22-06-01-44 +0000,latimes,CBS News correspondent accuses Bari Weiss of '...,https://www.latimes.com/entertainment-arts/bus...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
